In [1]:
# Training a Multi-label Classification Network using TF 2.0

In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras import optimizers, losses, metrics, callbacks
from tensorflow.keras.layers import Activation, Dense, Input, BatchNormalization

In [7]:
# define training and validation dataset path
train_path = "train.csv"
val_path = "val.csv"

In [8]:
# Read Training and Validation data
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)

In [10]:
train.head()

,customer_id,source_domain_normalized_vector,target_domain_ordered_categories
0,masked,"0.0, 0.0, 0.0, 0.0, 0.0, 0.161, 0.0, 0.0, 0.0,...","0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,masked,"0.0, 0.0, 0.039, 0.0, 0.0, 0.314, 0.0, 0.0, 0....","0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,masked,"0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0..."
3,masked,"0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0..."
4,masked,"0.0, 0.24, 0.0, 0.0, 0.0, 0.08, 0.04, 0.0, 0.0...","0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0..."


In [101]:
train['source_domain_normalized_vector'] = train['source_domain_normalized_vector'].apply(lambda x: eval(x))
train['target_domain_ordered_categories'] = train['target_domain_ordered_categories'].apply(lambda x: eval(x))

In [102]:
# Convert Dependent and Independent Variables to Pandas
X_train = pd.DataFrame(train['source_domain_normalized_vector'].to_list()).values
y_train = pd.DataFrame(train['target_domain_ordered_categories'].to_list()).values

In [103]:
X_train.shape, y_train.shape

((1000, 88), (1000, 34))

In [105]:
# Define Hyperparameters

lr = 0.1
epochs = 20
batch_size = 32

In [106]:
tf.keras.backend.clear_session()

input_dim = 88

model = Sequential()

model.add(Input(shape=(input_dim, )))
model.add(Dense(32, input_dim=input_dim, activation='relu'))
model.add(Dense(34, activation='sigmoid'))
          
model.compile(
    optimizer = optimizers.SGD(learning_rate = lr),
    loss = losses.BinaryCrossentropy(),
    metrics = [metrics.BinaryCrossentropy(), metrics.BinaryAccuracy()]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2848      
_________________________________________________________________
dense_1 (Dense)              (None, 34)                1122      
Total params: 3,970
Trainable params: 3,970
Non-trainable params: 0
_________________________________________________________________


In [107]:
model_name = f"multi_label_model_{epochs}_{batch_size}_{lr}"
model_name

'multi_label_model_20_32_0.1'

In [108]:
model_callbacks = [
    callbacks.EarlyStopping(patience=2),
    callbacks.ModelCheckpoint(filepath=f'./{model_name}/'),
    callbacks.TensorBoard(log_dir=f'./{model_name}/logs')
]

In [109]:
!ls

3 Multi Label L2 Category Classification DNN (1).ipynb
multi_label_model_20_32_0.1
requirements.txt
train.csv
train_2.csv
val.csv
val_2.csv


In [110]:
model.fit(
    X_train
    , y_train
    , epochs=epochs
    , batch_size=batch_size
    , use_multiprocessing=True
    , callbacks=model_callbacks 
)

Epoch 1/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6873 - binary_crossentropy: 0.6873 - binary_accuracy: 0.6138
INFO:tensorflow:Assets written to: ./multi_label_model_20_32_0.1/assets
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 0.6591 - binary_crossentropy: 0.6591 - binary_accuracy: 0.8749
INFO:tensorflow:Assets written to: ./multi_label_model_20_32_0.1/assets
Epoch 3/20
32/32 [==============================] - 0s 777us/step - loss: 0.6306 - binary_crossentropy: 0.6306 - binary_accuracy: 0.9195
INFO:tensorflow:Assets written to: ./multi_label_model_20_32_0.1/assets
Epoch 4/20
32/32 [==============================] - 0s 806us/step - loss: 0.5997 - binary_crossentropy: 0.5997 - binary_accuracy: 0.9328
INFO:tensorflow:Assets written to: ./multi_label_model_20_32_0.1/assets
Epoch 5/20
32/32 [==============================] - 0s 837us/step - loss: 0.5662 - binary_crossentropy: 0.5662 - binary_accuracy: 0.9367
INFO:tensorflow:Assets written 

In [111]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [112]:
%tensorboard --logdir ./multi_label_model_20_32_0.1/logs

Reusing TensorBoard on port 6007 (pid 13470), started 0:21:39 ago. (Use '!kill 13470' to kill it.)